In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from ultralytics import YOLO


In [25]:
model_path1 = '/tf/notebook/ElementDetect.pt'
model_path2 = '/tf/notebook/ElementDetect_swiper.pt'

weights1 = torch.load(model_path1)['model']
weights2 = torch.load(model_path2)['model']

print(weights1.keys())
print(weights2.keys())

# averaged_weights = {
#     key: (weights1[key] + weights2[key]) / 2
#     for key in weights1
#     if weights1[key] is not None and weights2[key] is not None
# }
averaged_weights = {
    key: (weights1[key] + weights2[key]) / 2
    for key in weights1.keys()
    if isinstance(weights1[key], torch.Tensor) and isinstance(weights2[key], torch.Tensor)
}
print(averaged_weights)

AttributeError: 'DetectionModel' object has no attribute 'keys'

In [1]:
def visualize_chest_imagenome_yolov8_predictions(
        model_name_or_path, dicom_id, image_size, conf_thres=0.25, iou_thres=0.45, figsize=(10, 10),
    ):

    # Load model
    from medvqa.models.vision.visual_modules import create_yolov8_model
    from medvqa.datasets.chest_imagenome import CHEST_IMAGENOME_NUM_BBOX_CLASSES, CHEST_IMAGENOME_BBOX_NAMES
    class_names = {i:x for i, x in enumerate(CHEST_IMAGENOME_BBOX_NAMES)}
    model = create_yolov8_model(
        model_name_or_path=model_name_or_path,
        nc=CHEST_IMAGENOME_NUM_BBOX_CLASSES,
        class_names=class_names,
    )

    # Load image
    from medvqa.datasets.mimiccxr import get_imageId2PartPatientStudy, get_mimiccxr_large_image_path
    imageId2PartPatientStudy = get_imageId2PartPatientStudy()    
    part_id, patient_id, study_id = imageId2PartPatientStudy[dicom_id]
    image_path = get_mimiccxr_large_image_path(part_id, patient_id, study_id, dicom_id)
    print(f'dicom_id = {dicom_id}')
    print(f'image_path = {image_path}')
    from medvqa.datasets.image_processing import get_image_transform
    transform = get_image_transform(
        image_size=image_size,
        use_bbox_aware_transform=True,
    )
    image, image_size_before, image_size_after = transform(image_path, return_image_size=True)
    print(f'image.shape = {image.shape}')
    print(f'image_size_before = {image_size_before}')
    print(f'image_size_after = {image_size_after}')

    # Run model in inference mode
    import torch
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    model.eval()
    with torch.no_grad():
        image = image.to(device)
        results = model(image.unsqueeze(0))
        assert len(results) == 2
        print(f'results[0].shape = {results[0].shape}')
        assert len(results[1]) == 3

    # Obtain predictions
    from ultralytics.yolo.utils.ops import non_max_suppression
    import numpy as np
    predictions = results[0].detach().cpu()
    predictions = non_max_suppression(predictions, conf_thres=conf_thres, iou_thres=iou_thres)
    print(f'len(predictions) (after NMS) = {len(predictions)}')
    predictions = predictions[0].numpy()
    print(f'predictions.shape = {predictions.shape}')
    pred_coords = predictions[:, :4]
    pred_coords /= np.array([image_size_before[1], image_size_before[0], image_size_before[1], image_size_before[0]])
    pred_classes = predictions[:, 5].astype(int)
    print(f'pred_coords.shape = {pred_coords.shape}')
    print(f'pred_classes.shape = {pred_classes.shape}')

    # Visualize predictions
    from medvqa.datasets.chest_imagenome.chest_imagenome_dataset_management import _visualize_predicted_bounding_boxes__yolo    
    _visualize_predicted_bounding_boxes__yolo(
        dicom_id=dicom_id,
        pred_coords=pred_coords,
        pred_classes=pred_classes,
        figsize=figsize,
        format='xyxy',
    )

    # Release GPU memory
    del model
    del image
    torch.cuda.empty_cache()
    return {
        'dicom_id': dicom_id,
        'pred_coords': pred_coords,
        'pred_classes': pred_classes,
    }

In [3]:
model_name_or_path = '/tf/notebook/ElementDetect.pt'

image_size = 640
visualize_chest_imagenome_yolov8_predictions()

TypeError: visualize_chest_imagenome_yolov8_predictions() missing 3 required positional arguments: 'model_name_or_path', 'dicom_id', and 'image_size'

In [10]:
from ultralytics import YOLO
def merge_detections(detections_model1, detections_model2, iou_threshold=0.5):
    merged_detections = detections_model1 + detections_model2
    
    final_detections = apply_nms(merged_detections, iou_threshold)
    return final_detections

detections_model1 = extract_detections(YOLO("/tf/notebook/ElementDetect.pt"), image='/tf/notebook/1231.jpg')
detections_model2 = extract_detections(YOLO("/tf/notebook/ElementDetect_swiper.pt"), image='/tf/notebook/1231.jpg')


final_detections = merge_detections(detections_model1, detections_model2)


FileNotFoundError: '<ultralytics.models.yolo.model.YOLO object at 0x7fc8f0fae970>' does not exist

In [9]:
def extract_detections(model_path, image):
    # 모델 로드
    model = YOLO(model_path)
    
    # 이미지 전처리
    preprocessed_image = preprocess_image(image)

    # 탐지 실행
    detections = model(preprocessed_image)

    # 결과 추출
    extracted_detections = []
    for detection in detections:
        # 여기서는 간단하게 bounding box, 클래스 ID, 신뢰도를 추출
        bbox = detection['bbox']
        class_id = detection['class_id']
        confidence = detection['confidence']
        extracted_detections.append((bbox, class_id, confidence))

    return extracted_detections


In [13]:
model1 = YOLO('/tf/notebook/runs/train17/weights/0912_step_1.pt')
model1.info()

Model summary: 295 layers, 25863847 parameters, 0 gradients


(295, 25863847, 0, 0.0)

In [14]:
metrics = model1.val()  # 데이터셋과 설정을 기억하니 인수는 필요 없습니다.
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps 

Ultralytics YOLOv8.0.149 🚀 Python-3.8.10 torch-2.0.1+cu117 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)
Model summary (fused): 218 layers, 25847287 parameters, 0 gradients
val: Scanning /tf/notebook/step1/val/labels.cache... 2447 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2447/2447 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 153/153 [00:22<00:00,  6.68it/s]
                   all       2447       5712      0.829      0.803      0.857      0.786
                banner       2447       1118      0.861      0.851      0.921       0.88
                2block       2447        411      0.749      0.742      0.805      0.752
                3block       2447        797      0.874      0.872      0.915      0.863
                4block       2447        908       0.88        0.9      0.941      0.892
                5block       2447        435      0.906      0.862      0.932      0.863
                6blo

array([    0.88046,     0.75241,     0.86297,     0.89152,     0.86335,     0.86283,     0.83463,     0.70179,     0.57984,      0.5445,     0.68352,     0.85451,     0.91035])

In [17]:
!yolo help
!yolo checks
!yolo version
!yolo settings
!yolo copy-cfg
!yolo cfg


    Arguments received: ['yolo', 'help']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of ('detect', 'segment', 'classify', 'pose')
                MODE (required) is one of ('train', 'val', 'predict', 'export', 'track', 'benchmark')
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train a detection model for 10 epochs with an initial learning_rate of 0.01
        yolo train data=coco128.yaml model=yolov8n.pt epochs=10 lr0=0.01

    2. Predict a YouTube video using a pretrained segmentation model at image size 320:
        yolo predict model=yolov8n-seg.pt source='https://youtu.be/Zgi9g1ksQHc' imgsz=320

    3. Val a pretrained detection model at batch-size 1 and image size 640:
        yolo val model=yolov8n.pt data=coco128.yaml batch=1 imgs